In [1]:
from preprocess_utils import *

large_hindex = 6
huge_hindex = 36

train, test = get_processed_data()
train = select_columns(train)
print(train.shape)

/tmp/ipykernel_8547/1550948794.py:6: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  train, test = get_processed_data()


(174241, 1033)


In [ ]:
train.head()

In [8]:
frequency = train["hindex"].value_counts(normalize=True)

# Small / Large Index Classification

In [4]:
from sklearn.model_selection import train_test_split

train_split, test_split = train_test_split(train)
print(train_split.shape, test_split.shape)

train_split.loc[train_split["hindex"] < large_hindex, "hindex"] = 0
test_split.loc[test_split["hindex"] < large_hindex, "hindex"] = 0
train_split.loc[train_split["hindex"] >= large_hindex, "hindex"] = 1
test_split.loc[test_split["hindex"] >= large_hindex, "hindex"] = 1

print(train_split.shape)

X_train = train_split.drop(
    ["author", "hindex"], axis=1
).to_numpy()
y_train = train_split["hindex"].to_numpy()
X_test = test_split.drop(
    ["author", "hindex"], axis=1
).to_numpy()
y_test = test_split["hindex"].to_numpy()

(130680, 41) (43561, 41)
(130680, 41)


In [5]:
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, f1_score, recall_score
model_cat = CatBoostClassifier(verbose=False)
model_cat.fit(X_train, y_train)
mod_preds = model_cat.predict(X_test)
print(classification_report(y_test, mod_preds))


0.8727761070682492
0.8741941342050304
0.8760236578707916


# Small / Large / Huge Index Classification

In [21]:
from sklearn.model_selection import train_test_split

train_split, test_split = train_test_split(train)
print(train_split.shape, test_split.shape)

train_split.loc[train_split["hindex"] < large_hindex, "hindex"] = 0
test_split.loc[test_split["hindex"] < large_hindex, "hindex"] = 0
train_split.loc[((train_split["hindex"] >= large_hindex) & (train_split["hindex"] < huge_hindex)), "hindex"] = 1
test_split.loc[((test_split["hindex"] >= large_hindex)  & (test_split["hindex"] < huge_hindex)), "hindex"] = 1
train_split.loc[train_split["hindex"] >= huge_hindex, "hindex"] = 2
test_split.loc[test_split["hindex"] >= huge_hindex, "hindex"] = 2

print(train_split.shape)

X_train = train_split.drop(
    ["author", "hindex"], axis=1
).to_numpy()
y_train = train_split["hindex"].to_numpy()
X_test = test_split.drop(
    ["author", "hindex"], axis=1
).to_numpy()
y_test = test_split["hindex"].to_numpy()

(130680, 41) (43561, 41)
(130680, 41)


In [22]:
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, f1_score, recall_score, classification_report
model_cat = CatBoostClassifier(verbose=False)
model_cat.fit(X_train, y_train)
mod_preds = model_cat.predict(X_test)
print(classification_report(y_test, mod_preds))

              precision    recall  f1-score   support

         0.0       0.87      0.87      0.87     21549
         1.0       0.81      0.85      0.83     19807
         2.0       0.75      0.47      0.58      2205

    accuracy                           0.84     43561
   macro avg       0.81      0.73      0.76     43561
weighted avg       0.84      0.84      0.84     43561



# Original Prediction MSE on large hindex

In [2]:
from catboost import CatBoostClassifier
from sklearn.metrics import train_test_split

train_split, test_split = train_test_split(train)
print(train_split.shape, test_split.shape)

X_train = train_split.drop(
    ["author", "hindex"], axis=1
).to_numpy()
y_train = train_split["hindex"].to_numpy()
X_test = test_split.drop(
    ["author", "hindex"], axis=1
).to_numpy()
y_test = test_split["hindex"].to_numpy()

from catboost import CatBoostRegressor
model_cat = CatBoostRegressor(verbose=False)
model_cat.fit(X_train, y_train)
mod_preds = model_cat.predict(X_test)
print("Overall MSE:", mean_squared_error(y_test, mod_preds))
y_big_test = [y for y in y_test if y >= large_hindex]
mod_big_preds = [mod_preds[i] for i in range(len(y_test)) if y_test[i] >= large_hindex]
print("Big Hindex MSE:", mean_squared_error(y_big_test, mod_big_preds))

(130680, 1033) (43561, 1033)


NameError: name 'mean_squared_error' is not defined

# Specific Prediction MSE on large hindex

In [40]:
big_train = train.loc[train["hindex"] >= large_hindex]

big_train_split, big_test_split = train_test_split(big_train)
print(big_train_split.shape, big_test_split.shape)

X_big_train = big_train_split.drop(
    ["author", "hindex"], axis=1
).to_numpy()
y_big_train = big_train_split["hindex"].to_numpy()
X_big_test = big_test_split.drop(
    ["author", "hindex"], axis=1
).to_numpy()
y_big_test = big_test_split["hindex"].to_numpy()

from catboost import CatBoostRegressor
model_cat = CatBoostRegressor(verbose=False)
model_cat.fit(X_big_train, y_big_train)
mod_preds = model_cat.predict(X_big_test)
print("Big Hindex MSE:", mean_squared_error(y_big_test, mod_preds))

(6469, 11) (2157, 11)
Big Hindex MSE: 170.16960484061605


# Original Prediction MSE on small hindex

In [43]:
train_split, test_split = train_test_split(train)
print(train_split.shape, test_split.shape)

X_train = train_split.drop(
    ["author", "hindex"], axis=1
).to_numpy()
y_train = train_split["hindex"].to_numpy()
X_test = test_split.drop(
    ["author", "hindex"], axis=1
).to_numpy()
y_test = test_split["hindex"].to_numpy()

from catboost import CatBoostRegressor
model_cat = CatBoostRegressor(verbose=False)
model_cat.fit(X_train, y_train)
mod_preds = model_cat.predict(X_test)
print("Overall MSE:", mean_squared_error(y_test, mod_preds))
y_small_test = [y for y in y_test if y < large_hindex]
mod_small_preds = [mod_preds[i] for i in range(len(y_test)) if y_test[i] < large_hindex]
print("Small Hindex MSE:", mean_squared_error(y_small_test, mod_small_preds))

(130680, 11) (43561, 11)
Overall MSE: 51.41873596298948
Small Hindex MSE: 25.906271543186012


# Specific Prediction MSE on small hindex

In [44]:
small_train = train.loc[train["hindex"] < large_hindex]

small_train_split, small_test_split = train_test_split(small_train)
print(small_train_split.shape, small_test_split.shape)

X_small_train = small_train_split.drop(
    ["author", "hindex"], axis=1
).to_numpy()
y_small_train = small_train_split["hindex"].to_numpy()
X_small_test = small_test_split.drop(
    ["author", "hindex"], axis=1
).to_numpy()
y_small_test = small_test_split["hindex"].to_numpy()

from catboost import CatBoostRegressor
model_cat = CatBoostRegressor(verbose=False)
model_cat.fit(X_small_train, y_small_train)
mod_preds = model_cat.predict(X_small_test)
print("Small Hindex MSE:", mean_squared_error(y_small_test, mod_preds))

(124211, 11) (41404, 11)
Small Hindex MSE: 18.99344045082073


# Combine Specific prediction and Large Index Classification

In [ ]:
from catboost import CatBoostRegressor, CatBoostClassifier
from sklearn.metrics import accuracy_score, f1_score, recall_score, mean_squared_error

large_hindex = 6

train_split, test_split = train_test_split(train)
train_split_classifier, test_split_classifier = train_split.copy(), test_split.copy()
print(train_split.shape, test_split.shape)

large_train_split = train_split.loc[train_split["hindex"] >= large_hindex]
large_test_split = test_split.loc[test_split["hindex"] >= large_hindex]
small_train_split = train_split.loc[train_split["hindex"] < large_hindex]
small_test_split = test_split.loc[test_split["hindex"] < large_hindex]

train_split_classifier.loc[train_split_classifier["hindex"] < large_hindex, "hindex"] = 0
train_split_classifier.loc[train_split_classifier["hindex"] >= large_hindex, "hindex"] = 1
test_split_classifier.loc[test_split_classifier["hindex"] < large_hindex, "hindex"] = 0
test_split_classifier.loc[test_split_classifier["hindex"] >= large_hindex, "hindex"] = 1

# Balance small and large data
# large_train_split_temp = large_train_split.append(small_train_split.sample(n=int(len(small_train_split) / 2.5), ignore_index=True))
# small_train_split = small_train_split.append(large_train_split_temp.sample(n=int(len(large_train_split) / 1.5), ignore_index=True))
# large_train_split = large_train_split_temp

print(large_train_split.shape, small_train_split.shape)

# Data for the classifier
X_train_classifier = train_split_classifier.drop(
    ["author", "hindex"], axis=1
).to_numpy()
y_train_classifier = train_split_classifier["hindex"].to_numpy()
X_test_classifier = test_split_classifier.drop(
    ["author", "hindex"], axis=1
).to_numpy()
y_test_classifier = test_split_classifier["hindex"].to_numpy()

# Data for large hindex regressor
X_large_train = large_train_split.drop(
    ["author", "hindex"], axis=1
).to_numpy()
y_large_train = large_train_split["hindex"].to_numpy()
X_large_test = large_test_split.drop(
    ["author", "hindex"], axis=1
).to_numpy()
y_large_test = large_test_split["hindex"].to_numpy()

# Data for small hindex regressor
X_small_train = small_train_split.drop(
    ["author", "hindex"], axis=1
).to_numpy()
y_small_train = small_train_split["hindex"].to_numpy()
X_small_test = small_test_split.drop(
    ["author", "hindex"], axis=1
).to_numpy()
y_small_test = small_test_split["hindex"].to_numpy()

# Original Data
X_train = train_split.drop(
    ["author", "hindex"], axis=1
).to_numpy()
y_train = train_split["hindex"].to_numpy()
X_test = test_split.drop(
    ["author", "hindex"], axis=1
).to_numpy()
y_test = test_split["hindex"].to_numpy()


model = CatBoostRegressor(verbose=False)
model.fit(X_train, y_train)
preds = model.predict(X_test)
print("Classical Hindex MSE:", mean_squared_error(y_test, preds))
print("Classical Hindex MSE on small:", mean_squared_error(y_small_test, model.predict(X_small_test)))
print("Classical Hindex MSE on large:", mean_squared_error(y_large_test, model.predict(X_large_test)))


model_small = CatBoostRegressor(verbose=False)
model_small.fit(X_small_train, y_small_train)
small_preds = model_small.predict(X_test)
print("Small Hindex MSE:", mean_squared_error(y_test, small_preds))
print("Small Hindex MSE on small:", mean_squared_error(y_small_test, model_small.predict(X_small_test)))
print("Small Hindex MSE on large:", mean_squared_error(y_large_test, model_small.predict(X_large_test)))


model_large = CatBoostRegressor(verbose=False)
model_large.fit(X_large_train, y_large_train)
large_preds = model_large.predict(X_test)
print("Large Hindex MSE:", mean_squared_error(y_test, large_preds))
print("Large Hindex MSE on small:", mean_squared_error(y_small_test, model_large.predict(X_small_test)))
print("Large Hindex MSE on large:", mean_squared_error(y_large_test, model_large.predict(X_large_test)))


hindex_classifier = CatBoostClassifier(verbose=False, num_trees=2000)
hindex_classifier.fit(X_train_classifier, y_train_classifier)
hindex_classifier_preds = hindex_classifier.predict(X_test_classifier)
print(accuracy_score(y_test_classifier, hindex_classifier_preds), f1_score(y_test_classifier,hindex_classifier_preds), recall_score(y_test_classifier,hindex_classifier_preds))

hindex_classifier_probas = hindex_classifier.predict_proba(X_test_classifier)

combine_preds = []
count_small, count_large, count_classic = 0, 0, 0
count_small_correct, count_large_correct = 0, 0

threshold = 0.95
for i in range(len(y_test)):
    # if hindex_classifier_preds == 0:
    #     combine_preds.append(small_preds[i])
    # else:
    #     combine_preds.append(large_preds[i])
    # if hindex_classifier_probas[i][0] > threshold:
    #     combine_preds.append(small_preds[i])
    #     count_small += 1
    #     if y_test_classifier[i] == 0:
    #         count_small_correct += 1
    # elif hindex_classifier_probas[i][1] > threshold:
    #     combine_preds.append(large_preds[i])
    #     count_large += 1
    #     if y_test_classifier[i] == 1:
    #         count_large_correct += 1
    # else:
    #     combine_preds.append(preds[i])
    #     count_classic += 1
    combine_preds.append((hindex_classifier_probas[i][0] * small_preds[i] + hindex_classifier_probas[i][1] * large_preds[i] + preds[i]) / 2)

print("Combine Hindex MSE:", mean_squared_error(y_test, combine_preds))
# print("count :", count_small, count_large, count_classic)
# print("count correct:", count_small_correct, count_large_correct)
# print("ratio correct:", count_small_correct/count_small, count_large_correct/count_large)

In [ ]:
from catboost import CatBoostRegressor, CatBoostClassifier
from sklearn.metrics import classification_report, mean_squared_error
import math


def get_small_data(train, test, large_hindex):
    small_train = train.loc[train_split["hindex"] < large_hindex]
    small_test = test.loc[test_split["hindex"] < large_hindex]

    # Data for small hindex regressor
    X_small_train = small_train.drop(["author", "hindex"], axis=1).to_numpy()
    y_small_train = small_train["hindex"].to_numpy()
    X_small_test = small_test.drop(["author", "hindex"], axis=1).to_numpy()
    y_small_test = small_test["hindex"].to_numpy()

    return X_small_train, y_small_train, X_small_test, y_small_test


def get_large_data(train, test, large_hindex, huge_hindex):
    large_train = train.loc[
        (train_split["hindex"] >= large_hindex) & (train_split["hindex"] < huge_hindex)
    ]
    large_test = test.loc[
        (test_split["hindex"] >= large_hindex) & (test_split["hindex"] < huge_hindex)
    ]

    # Data for large hindex regressor
    X_large_train = large_train.drop(["author", "hindex"], axis=1).to_numpy()
    y_large_train = large_train["hindex"].to_numpy()
    X_large_test = large_test.drop(["author", "hindex"], axis=1).to_numpy()
    y_large_test = large_test["hindex"].to_numpy()

    return X_large_train, y_large_train, X_large_test, y_large_test


def get_huge_data(train, test, huge_hindex):
    huge_train = train.loc[train_split["hindex"] >= huge_hindex]
    huge_test = test.loc[test_split["hindex"] >= huge_hindex]

    # Data for huge hindex regressor
    X_huge_train = huge_train.drop(["author", "hindex"], axis=1).to_numpy()
    y_huge_train = huge_train["hindex"].to_numpy()
    X_huge_test = huge_test.drop(["author", "hindex"], axis=1).to_numpy()
    y_huge_test = huge_test["hindex"].to_numpy()

    return X_huge_train, y_huge_train, X_huge_test, y_huge_test


def get_classifier_data(train, test):
    train = train.copy()
    test = test.copy()
    
    train.loc[train_split["hindex"] < large_hindex, "hindex"] = 0
    test.loc[test_split["hindex"] < large_hindex, "hindex"] = 0
    train.loc[((train_split["hindex"] >= large_hindex) & (train_split["hindex"] < huge_hindex)), "hindex"] = 1
    test.loc[((test_split["hindex"] >= large_hindex)  & (test_split["hindex"] < huge_hindex)), "hindex"] = 1
    train.loc[train_split["hindex"] >= huge_hindex, "hindex"] = 2
    test.loc[test_split["hindex"] >= huge_hindex, "hindex"] = 2

    # Data for the classifier
    X_train_classifier = train_split_classifier.drop(
        ["author", "hindex"], axis=1
    ).to_numpy()
    y_train_classifier = train_split_classifier["hindex"].to_numpy()
    X_test_classifier = test_split_classifier.drop(
        ["author", "hindex"], axis=1
    ).to_numpy()
    y_test_classifier = test_split_classifier["hindex"].to_numpy()

    return X_train_classifier, y_train_classifier, X_test_classifier, y_test_classifier

def get_original_data(train, test):
    # Original Data
    X_train = train_split.drop(
        ["author", "hindex"], axis=1
    ).to_numpy()
    y_train = train_split["hindex"].to_numpy()
    X_test = test_split.drop(
        ["author", "hindex"], axis=1
    ).to_numpy()
    y_test = test_split["hindex"].to_numpy()

    return X_train, y_train, X_test, y_test

# Duplicate very large hindex

In [79]:
from catboost import CatBoostRegressor, CatBoostClassifier
from sklearn.metrics import accuracy_score, f1_score, recall_score, mean_squared_error

very_large_hindex = 36

train_split, test_split = train_test_split(train)

very_large_train_split = train_split.loc[train_split["hindex"] >= very_large_hindex]

train_split = train_split.append(very_large_train_split)
print(train_split.shape)

X_train = train_split.drop(
    ["author", "hindex"], axis=1
).to_numpy()
y_train = train_split["hindex"].to_numpy()
X_test = test_split.drop(
    ["author", "hindex"], axis=1
).to_numpy()
y_test = test_split["hindex"].to_numpy()


model = CatBoostRegressor(verbose=False)
model.fit(X_train, y_train)
preds = model.predict(X_test)
print("Classical Hindex MSE:", mean_squared_error(y_test, preds))


(130680, 11)
Classical Hindex MSE: 51.031006030063494


# Fasttext Classification

In [3]:
import numpy as np
from preprocess_utils import df_to_txt

large_hindex = 6
train_fasttext, test_fasttext = get_processed_data()
train_fasttext["hindex_lab"] = np.where(train_fasttext["hindex"] >= large_hindex, "__label__1", "__label__0")

train_fasttext_split, test_fasttext_split = train_test_split(train_fasttext)

path_fasttext_text = "../tmp/fasttext_text.txt"
df_to_txt(train_fasttext, path_fasttext_text)
model_fasttext = fasttext.train_supervised(
        path_fasttext_text, lr=0.15815, dim=2, epoch=33, wordNgrams=3
)
os.remove(path_fasttext_text)

/tmp/ipykernel_6653/626770302.py:5: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  train_fasttext, test_fasttext = get_processed_data()
Read 99M words
Number of words:  359997
Number of labels: 2
Progress: 100.0% words/sec/thread: 5831801 lr:  0.000000 avg.loss:  0.556754 ETA:   0h 0m 0s


In [4]:
path_fasttext_text = "../tmp/fasttext_text.txt"
df_to_txt(test_fasttext_split, path_fasttext_text)
print(model_fasttext.test(path_fasttext_text))
os.remove(path_fasttext_text)

(43561, 0.7491104428272997, 0.7491104428272997)


## Add Fasttext prediction to data

In [33]:
preds = model_fasttext.predict(train_fasttext["text"])

AttributeError: 'Series' object has no attribute 'find'

In [19]:
train.head()

,author,hindex,nb_paper,pagerank,authority,clustering_coef,n_neighbors_dist_1,mean_neighbors_dist_1,max_neighbors_dist_1,vector_coord_0,vector_coord_1,fasttext_pred
0,1964267543,4.0,5,0.000004,3.339138e-13,1.000000,5,21.800000,39,-0.186604,0.288065,__label__1
1,2153592714,13.0,5,0.000002,1.433035e-17,1.000000,2,20.000000,20,-0.239073,0.446518,__label__0
2,217158525,8.0,5,0.000003,6.750288e-20,1.000000,2,2.000000,2,-0.225883,0.357737,__label__1
3,2123103677,11.0,3,0.000005,7.714573e-16,0.714286,7,11.833333,54,-0.174987,0.208499,__label__1
4,2067710487,3.0,2,0.000004,-1.444459e-26,0.000000,2,9.841160,12,-0.170414,0.257662,__label__1


In [28]:
from tqdm import tqdm
import pandas as pd

fasttext_pred = []
for text in tqdm(train_fasttext["text"].to_list()):
    if not pd.isna(text):
        label, proba = model_fasttext.predict(text)
    else:
        label, proba = model_fasttext.predict("")
    if label[0] == "__label__1":
        fasttext_pred.append(proba[0])
    else: 
        fasttext_pred.append(1-proba[0])

100%|██████████| 174241/174241 [00:14<00:00, 11636.55it/s]


In [29]:
train["fasttext_pred"] = fasttext_pred